## Pismo - Case para vaga de Analytics Engineer III
#### Autor: Edson Delavia

## Contexto
Foi solicitado um crawler para puxar as cotações históricas de algumas moedas, persistir as informações em alguma base relacional, cruzar com um csv que apresenta os valores históricos do café e disponibilizar as seguintes informações:
 - Maior volume negociado de café no dia e as cotações de fechamento
 - Total de café negociado por ano e as cotações
 - Média de volume negociado mensal e anual
 - Um dash apresentando em formato big number a última cotação do café em diferentes moedas e o histórico de variação do preço do café ao longo do tempo.

## Setup
para facilitar a testabilidade, foi criado um script de setup que automatiza as seguintes tarefas:
- Cria um ambiente virtual em python3
- Instala as bibliotecas necessárias
- Cria o banco em sqlite e o schema para receber as informações depois
- Persiste no banco as informações do csv coffee.csv, localizado na pasta "entries"
- Roda o crawler com informações de 2020 à 2022-08-31 e persiste as informações no banco, no esquema de merge (se a informação existir, será sobrescrita)

O processo todo demora em média 23 minutos (resultados mensurados após uma bateria de 11 testes), grande parte do tempo é para instalar as libs, principalmente pandas e jupyter.

Para rodar o setup, basta rodar a célula abaixo, o %%capture serve para as mensagens de log não serem apresentadas no output da célula, por apresentar informações demais e poluir o notebook

In [11]:
%%capture

! source setup.sh

## Consultas

In [19]:
%load_ext sql
%sql sqlite:///quotes.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [59]:
import pandas as pd

### Maior volume negociado de café no dia e as cotações de fechamento

Obs: aqui tomei a liberdade para já converter o valor do café nas cotações de cada país

In [51]:
%%sql

-- aqui pivotei usando o filter, na segunda questão eu uso a forma de CASE WHEN, apenas para mostrar duas formas diferentes mesmo
-- já que sqlite não tem a função PIVOT nativa

SELECT
    coffee.Date as DATE,
    coffee.High as COFFEE_USD,
    round(coffee.High * max(quotes.value) filter(where quotes.currency = 'USDBRL'), 2) as COFFEE_BRL,
    round(coffee.High * max(quotes.value) filter(where quotes.currency = 'USDCLP'), 2) as COFFEE_CLP,
    round(coffee.High * max(quotes.value) filter(where quotes.currency = 'USDEUR'), 2) as COFFEE_EUR
FROM
    coffee
INNER JOIN  
    daily_quotes as quotes on quotes.date = coffee.Date
GROUP BY
    coffee.Date,
    coffee.High
LIMIT 10

 * sqlite:///quotes.db
Done.


DATE,COFFEE_USD,COFFEE_BRL,COFFEE_CLP,COFFEE_EUR
2019-01-02,102.65,389.03,71528.63,90.7
2019-01-03,103.25,387.87,71634.76,90.65
2019-01-04,103.0,382.67,70164.0,90.39
2019-01-07,103.45,386.45,70475.27,90.12
2019-01-08,106.05,393.86,72156.48,92.57
2019-01-09,106.85,393.37,72230.61,92.47
2019-01-10,106.0,393.41,71666.31,92.12
2019-01-11,104.8,388.96,70729.93,91.37
2019-01-14,103.55,383.03,69761.33,90.26
2019-01-15,102.9,382.57,69509.17,90.15


### Total de café negociado por ano e as cotações

In [52]:
%%sql

WITH

coffe_volume_by_year as (
SELECT
    strftime('%Y', Date) as YEAR,
    sum(Volume) as COFFEE_TOTAL_VOLUME
FROM
    coffee
GROUP BY
    strftime('%Y', Date)
),

quotes_yearly_closes as (
SELECT
    date,
    row_number() over (partition by currency, strftime('%Y', date) order by date desc) as last_date_of_year,
    currency,
    value
FROM
    daily_quotes
),

pivoted_yearly_closes as (
SELECT
    strftime('%Y', date) as YEAR,
    date as QUOTE_REF_DATE,
    round(avg(case when currency = 'USDBRL' then value end), 4) as USDBRL,
    round(avg(case when currency = 'USDCLP' then value end), 4) as USDCLP,
    round(avg(case when currency = 'USDEUR' then value end), 4) as USDEUR
FROM
    quotes_yearly_closes
WHERE
    last_date_of_year = 1
GROUP BY
    strftime('%Y', date),
    date
),

result as (
SELECT
    coffee.YEAR,
    coffee.COFFEE_TOTAL_VOLUME,
    quotes.USDBRL,
    quotes.USDCLP,
    quotes.USDEUR
FROM
    coffe_volume_by_year as coffee
INNER JOIN 
    pivoted_yearly_closes as quotes using (YEAR)
)

select * from result

 * sqlite:///quotes.db
Done.


YEAR,COFFEE_TOTAL_VOLUME,USDBRL,USDCLP,USDEUR
2019,4011645,4.02,739.4031,0.8913
2020,3078378,5.1936,710.5025,0.8211
2021,3127536,5.5713,852.0004,0.8794
2022,2374254,5.1833,897.0002,0.9961


### Média de volume negociado mensal e anual

In [58]:
%%sql

SELECT DISTINCT
    strftime('%Y', Date) as YEAR,
    strftime('%Y-%m', Date) as YEARMONTH,
    round(avg(Volume) over (partition by strftime('%Y', Date)),  0) as YEAR_AVG_VOLUME,
    round(avg(Volume) over (partition by strftime('%Y-%m', Date)), 0) as MONTH_AVG_VOLUME
FROM
    coffee
LIMIT 12

 * sqlite:///quotes.db
Done.


YEAR,YEARMONTH,YEAR_AVG_VOLUME,MONTH_AVG_VOLUME
2019,2019-01,15919.0,21827.0
2019,2019-02,15919.0,17393.0
2019,2019-03,15919.0,8917.0
2019,2019-04,15919.0,23584.0
2019,2019-05,15919.0,13223.0
2019,2019-06,15919.0,22038.0
2019,2019-07,15919.0,8820.0
2019,2019-08,15919.0,15226.0
2019,2019-09,15919.0,7426.0
2019,2019-10,15919.0,24074.0


### Dashboard

Para elaborar o dashboard, vamos usar a mesma base da primeira questão, mas pegaremos apenas as cotações do último dia dos meses fechados. Assim conseguimos ver uma visão de fechamento mensal sem a variabilidade diária que costuma poluir algumas análises. Os resultados serão salvos na pasta "results".

In [72]:
%%sql

dashboard_results <<

WITH

pivoted_data as (
SELECT
    coffee.Date as DATE,
    coffee.High as COFFEE_USD,
    round(coffee.High * max(quotes.value) filter(where quotes.currency = 'USDBRL'), 2) as COFFEE_BRL,
    round(coffee.High * max(quotes.value) filter(where quotes.currency = 'USDCLP'), 2) as COFFEE_CLP,
    round(coffee.High * max(quotes.value) filter(where quotes.currency = 'USDEUR'), 2) as COFFEE_EUR
FROM
    coffee
INNER JOIN  
    daily_quotes as quotes on quotes.date = coffee.Date
GROUP BY
    coffee.Date,
    coffee.High
),

month_closes_flag as (
SELECT
    *,
    row_number() over (partition by strftime("%Y-%m", DATE) order by DATE desc) = 1 is_last_date_of_month
FROM
    pivoted_data
),

result as (
SELECT
    DATE,
    COFFEE_USD,
    COFFEE_BRL,
    COFFEE_CLP,
    COFFEE_EUR
FROM
    month_closes_flag
WHERE
    is_last_date_of_month = True
)

select * from result

 * sqlite:///quotes.db
Done.
Returning data to local variable dashboard_results


In [78]:
dash_df = dashboard_results.DataFrame()

dash_df.to_csv('results/dashboard_results.csv', sep = ';', encoding = 'utf8')

dash_df[:10]

,DATE,COFFEE_USD,COFFEE_BRL,COFFEE_CLP,COFFEE_EUR
0,2019-01-31,106.35,387.57,70474.96,92.91
1,2019-02-28,95.55,358.41,62661.19,84.01
2,2019-03-29,95.65,375.24,65090.20,85.16
3,2019-04-30,93.85,367.74,63601.92,83.68
4,2019-05-31,105.30,413.31,74729.35,94.03
5,2019-06-28,109.00,419.76,73847.93,95.70
6,2019-07-31,100.45,383.04,70677.12,90.73
7,2019-08-30,93.30,386.77,67295.89,84.74
8,2019-09-30,102.45,425.81,74647.09,94.00
9,2019-10-31,102.25,410.79,76084.64,91.65


O dashboard pode ser acessado através do link https://datastudio.google.com/s/gqhqJWTFuKY, mas também foi incorporado abaixo no notebook

<iframe width="600" height="750" src="https://datastudio.google.com/embed/reporting/604bcdc5-81df-45ab-abf4-ddd152bd9ddc/page/g7v5C" frameborder="0" style="border:0" allowfullscreen></iframe>